ekstraktowanie cech związanych z zabudową

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import MultiPolygon, MultiPoint
from scipy.spatial.distance import cdist
import numpy as np

In [2]:
# pobranie granic miejscowości już po usunięciu tych bez własnej adrsacji
granice_miejscowosci_z_adresacja = gpd.read_file('28/warminskie_granice_miejscowosci_z_adresacja.shp')
granice_miejscowosci_z_adresacja

,TERYT,LOKALNYID,WERSJAID,PNAZW,GMLID,X_KOD,X_SKRKARTO,X_SKRK_NIL,X_KATDOKG,X_DOKLGEOM,...,EM_LOKID,EM_PNAZW,EM_WERID,PRNG_LOKID,PRNG_PNAZW,PRNG_WERID,area,layer,path,geometry
0,2801,2C98CCB7-E677-AAFE-E053-CA2BA8C0B325,2022-03-01T12:46:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.0177,ADMS03,NaN,inapplicable,dokladny,NaN,...,0474382,PL.ZIPIN.870,NaN,109502,PL.PZGiK.204.PRNG,NaN,2773713.13,PL.PZGiK.341.2801__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((602428.650 708821.710, 602501.620 70..."
1,2801,2C98CCB7-E663-AAFE-E053-CA2BA8C0B325,2022-03-01T12:46:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.0171,ADMS03,NaN,inapplicable,dokladny,NaN,...,0474324,PL.ZIPIN.870,NaN,99197,PL.PZGiK.204.PRNG,NaN,6744120.39,PL.PZGiK.341.2801__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((600581.150 708206.820, 600756.320 70..."
2,2801,2C98CCB7-E68D-AAFE-E053-CA2BA8C0B325,2022-03-01T12:46:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.0196,ADMS03,NaN,inapplicable,dokladny,NaN,...,0474442,PL.ZIPIN.870,NaN,126609,PL.PZGiK.204.PRNG,NaN,7996737.92,PL.PZGiK.341.2801__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((597062.200 717054.300, 597160.520 71..."
3,2801,2C98CCB7-E690-AAFE-E053-CA2BA8C0B325,2022-03-01T12:46:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.01108,ADMS03,NaN,inapplicable,dokladny,NaN,...,0474465,PL.ZIPIN.870,NaN,140116,PL.PZGiK.204.PRNG,NaN,3882361.42,PL.PZGiK.341.2801__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((602589.080 723526.660, 602670.420 72..."
4,2801,2C98CCB7-E66F-AAFE-E053-CA2BA8C0B325,2022-03-01T12:46:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.01228,ADMS07,NaN,inapplicable,przyblizony,NaN,...,0474420,PL.ZIPIN.870,NaN,127660,PL.PZGiK.204.PRNG,NaN,129325.10,PL.PZGiK.341.2801__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((599604.680 721448.040, 599629.000 72..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3768,2819,311304D1-499A-B2CD-E053-CC2BA8C0FE3A,2021-10-06T13:36:00,PL.PZGiK.341.BDOT10k,OT_ADMS_A.104,ADMS03,NaN,inapplicable,dokladny,1.0,...,765116,PL.ZIPIN.1943,NaN,00000000-0000-0000-0000-000000040858,PL.PZGiK.204.PRNG,NaN,47631769.37,PL.PZGiK.341.2819__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((682986.450 703211.410, 682991.700 70..."
3769,2819,311304D1-4996-B2CD-E053-CC2BA8C0FE3A,2021-10-06T13:36:00,PL.PZGiK.341.BDOT10k,OT_ADMS_A.103,ADMS03,NaN,inapplicable,dokladny,1.0,...,770979,PL.ZIPIN.903,NaN,00000000-0000-0000-0000-000000040171,PL.PZGiK.204.PRNG,NaN,22742342.71,PL.PZGiK.341.2819__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((667907.680 715869.080, 667852.960 71..."
3770,2819,311304D1-49C6-B2CD-E053-CC2BA8C0FE3A,2021-10-06T13:36:00,PL.PZGiK.341.BDOT10k,OT_ADMS_A.116,ADMS07,NaN,inapplicable,dokladny,1.0,...,771341,PL.ZIPIN.903,NaN,00000000-0000-0000-0000-000000133740,PL.PZGiK.204.PRNG,NaN,3079021.72,PL.PZGiK.341.2819__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((669232.280 706589.380, 669260.150 70..."
3771,2819,311304D1-49D7-B2CD-E053-CC2BA8C0FE3A,2021-10-06T13:36:00,PL.PZGiK.341.BDOT10k,OT_ADMS_A.47,ADMS07,NaN,inapplicable,dokladny,1.0,...,755632,PL.ZIPIN.902,NaN,00000000-0000-0000-0000-000000026309,PL.PZGiK.204.PRNG,NaN,2833038.18,PL.PZGiK.341.2819__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((682807.210 712243.900, 682802.600 71..."


In [3]:
# pobranie budynków
budynki = gpd.read_file('28/warminskie_budynki.shp')
budynki

,TERYT,LOKALNYID,WERSJAID,PNAZW,GMLID,X_KOD,X_SKRKARTO,X_SKRK_NIL,X_KATDOKG,X_DOKLGEOM,...,FUNSZC_NIL,KODKST,LKOND,LKOND_NIL,NAZWA,ZABYTEK,area,layer,path,geometry
0,2801,2C98CCB7-EB4A-AAFE-E053-CA2BA8C0B325,2020-09-25T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.BUBDA.28.012304,BUBD01,NaN,inapplicable,dokladny,NaN,...,NaN,110,2.0,NaN,NaN,Nie,85.60,PL.PZGiK.341.2801__OT_BUBD_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((610883.070 711181.190, 610885.070 71..."
1,2801,fa78a974-327d-4dea-853e-89f282afae2e,2020-09-25T00:00:00,PL.PZGiK.341.BDOT10k,OT_BUBD_A.21f9dcf8-c18e-4554-aa45-35c00d10e1aa,BUBD18,NaN,inapplicable,dokladny,NaN,...,NaN,108,1.0,NaN,NaN,Nie,98.33,PL.PZGiK.341.2801__OT_BUBD_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((600636.810 722210.250, 600631.300 72..."
2,2801,2C98CCB8-24F9-AAFE-E053-CA2BA8C0B325,2020-09-25T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.BUBDA.28.0117042,BUBD10,gar.,NaN,dokladny,NaN,...,NaN,102,1.0,NaN,NaN,Nie,77.22,PL.PZGiK.341.2801__OT_BUBD_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((622995.510 707641.790, 622982.050 70..."
3,2801,a39790b1-6cf4-11e9-83ab-5404a6a68038,2020-09-25T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.BUBDA.28.100_21367,BUBD18,NaN,inapplicable,dokladny,NaN,...,NaN,108,1.0,NaN,NaN,Nie,98.24,PL.PZGiK.341.2801__OT_BUBD_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((604178.290 718508.330, 604173.100 71..."
4,2801,9A460CD3-6BA8-4D5F-B30F-D552A3830AC4,2020-10-03T00:00:00,PL.PZGiK.341.BDOT10k,ID_9A460CD3-6BA8-4D5F-B30F-D552A3830AC4,BUBD01,NaN,inapplicable,dokladny,NaN,...,NaN,110,2.0,NaN,NaN,Nie,90.72,PL.PZGiK.341.2801__OT_BUBD_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((587505.660 723024.460, 587516.310 72..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514959,2862,2BF83540-A5EB-83A2-E053-CC2BA8C04E93,2022-08-08T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.BUBDA.28.6212187,BUBD21,pomp.,NaN,dokladny,NaN,...,NaN,109,2.0,NaN,NaN,Nie,1341.46,PL.PZGiK.341.2862__OT_BUBD_A,C:\studia\pracka\28_SHP\28\2862_SHP\PL.PZGiK.3...,"POLYGON ((595479.260 654416.380, 595498.500 65..."
514960,2862,2BF83540-9219-83A2-E053-CC2BA8C04E93,2022-08-08T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.BUBDA.28.627015,BUBD08,NaN,inapplicable,dokladny,NaN,...,NaN,103,1.0,NaN,NaN,Nie,152.82,PL.PZGiK.341.2862__OT_BUBD_A,C:\studia\pracka\28_SHP\28\2862_SHP\PL.PZGiK.3...,"POLYGON ((595247.990 657244.210, 595244.510 65..."
514961,2862,2BF83540-82C8-83A2-E053-CC2BA8C04E93,2022-08-08T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.BUBDA.28.622599,BUBD21,NaN,inapplicable,dokladny,NaN,...,NaN,109,1.0,NaN,NaN,Nie,115.84,PL.PZGiK.341.2862__OT_BUBD_A,C:\studia\pracka\28_SHP\28\2862_SHP\PL.PZGiK.3...,"POLYGON ((594989.630 660793.400, 595000.860 66..."
514962,2862,2BF83540-8974-83A2-E053-CC2BA8C04E93,2022-08-08T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.BUBDA.28.625046,BUBD21,NaN,inapplicable,dokladny,NaN,...,NaN,109,1.0,NaN,NaN,Nie,53.50,PL.PZGiK.341.2862__OT_BUBD_A,C:\studia\pracka\28_SHP\28\2862_SHP\PL.PZGiK.3...,"POLYGON ((598415.940 656333.760, 598417.970 65..."


In [4]:
# ograniczenie warstwy granaic tylko do dwóch kolumn
granice_temp = granice_miejscowosci_z_adresacja[['SIMC_id', 'geometry']]
granice_temp

,SIMC_id,geometry
0,474382,"POLYGON ((602428.650 708821.710, 602501.620 70..."
1,474324,"POLYGON ((600581.150 708206.820, 600756.320 70..."
2,474442,"POLYGON ((597062.200 717054.300, 597160.520 71..."
3,474465,"POLYGON ((602589.080 723526.660, 602670.420 72..."
4,474420,"POLYGON ((599604.680 721448.040, 599629.000 72..."
...,...,...
3768,765116,"POLYGON ((682986.450 703211.410, 682991.700 70..."
3769,770979,"POLYGON ((667907.680 715869.080, 667852.960 71..."
3770,771341,"POLYGON ((669232.280 706589.380, 669260.150 70..."
3771,755632,"POLYGON ((682807.210 712243.900, 682802.600 71..."


In [5]:
# pogrupowanie budynków wg tego w jakiej miejscowości (granicy miejscowości się znajdują)
budynki_grouped = gpd.sjoin(budynki, granice_temp, how='inner', op="within")
budynki_grouped.drop_duplicates(inplace=True)
budynki_grouped

C:\Users\agnie\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,TERYT,LOKALNYID,WERSJAID,PNAZW,GMLID,X_KOD,X_SKRKARTO,X_SKRK_NIL,X_KATDOKG,X_DOKLGEOM,...,LKOND,LKOND_NIL,NAZWA,ZABYTEK,area,layer,path,geometry,index_right,SIMC_id
0,2801,2C98CCB7-EB4A-AAFE-E053-CA2BA8C0B325,2020-09-25T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.BUBDA.28.012304,BUBD01,NaN,inapplicable,dokladny,NaN,...,2.0,NaN,NaN,Nie,85.60,PL.PZGiK.341.2801__OT_BUBD_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((610883.070 711181.190, 610885.070 71...",83,470332
135,2801,6A2F9B96-E76E-4732-82D9-5B070EDE755D,2020-11-16T00:00:00,PL.PZGiK.341.BDOT10k,ID_6A2F9B96-E76E-4732-82D9-5B070EDE755D,BUBD18,NaN,inapplicable,dokladny,NaN,...,1.0,NaN,NaN,Nie,44.61,PL.PZGiK.341.2801__OT_BUBD_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((611833.290 711889.240, 611837.160 71...",83,470332
195,2801,809473e1-9bfb-11e9-a4d3-b8ca3a7f05dc,2020-09-25T00:00:00,PL.PZGiK.341.BDOT10k,OT_BUBD_A_1561984007_43,BUBD18,NaN,inapplicable,przyblizony,NaN,...,1.0,NaN,NaN,Nie,114.15,PL.PZGiK.341.2801__OT_BUBD_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((611186.630 710793.390, 611193.890 71...",83,470332
505,2801,7b14dd53-3c61-11e9-96d2-f01faf546c5e,2020-09-25T00:00:00,PL.PZGiK.341.BDOT10k,OT_BUBD_A_1551472495_86,BUBD18,NaN,inapplicable,dokladny,NaN,...,1.0,NaN,NaN,Nie,53.13,PL.PZGiK.341.2801__OT_BUBD_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((611894.910 712055.640, 611896.890 71...",83,470332
601,2801,2C98CCB7-F8BA-AAFE-E053-CA2BA8C0B325,2020-09-25T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.BUBDA.28.013355,BUBD18,NaN,inapplicable,dokladny,NaN,...,1.0,NaN,NaN,Nie,44.73,PL.PZGiK.341.2801__OT_BUBD_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((610826.210 711047.120, 610827.520 71...",83,470332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480985,2819,311304D1-6A60-B2CD-E053-CC2BA8C0FE3A,2020-09-07T12:00:00,PL.PZGiK.341.BDOT10k,OT_BUBD_A.3817,BUBD18,NaN,inapplicable,dokladny,1.0,...,1.0,NaN,NaN,Nie,204.87,PL.PZGiK.341.2819__OT_BUBD_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((679715.360 719748.060, 679715.400 71...",3723,755490
482496,2819,311304D1-514C-B2CD-E053-CC2BA8C0FE3A,2020-09-07T12:00:00,PL.PZGiK.341.BDOT10k,OT_BUBD_A.8263,BUBD01,NaN,inapplicable,dokladny,1.0,...,1.0,NaN,NaN,Nie,122.18,PL.PZGiK.341.2819__OT_BUBD_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((679731.330 719578.050, 679731.370 71...",3723,755490
483727,2819,311304D1-5F83-B2CD-E053-CC2BA8C0FE3A,2020-09-07T12:00:00,PL.PZGiK.341.BDOT10k,OT_BUBD_A.8111,BUBD01,NaN,inapplicable,dokladny,1.0,...,1.0,NaN,NaN,Nie,184.27,PL.PZGiK.341.2819__OT_BUBD_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((679719.040 719771.710, 679719.110 71...",3723,755490
477442,2819,311304D1-64E7-B2CD-E053-CC2BA8C0FE3A,2020-09-07T12:00:00,PL.PZGiK.341.BDOT10k,OT_BUBD_A.10868,BUBD01,NaN,inapplicable,dokladny,1.0,...,1.0,NaN,NaN,Nie,127.71,PL.PZGiK.341.2819__OT_BUBD_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((669341.210 706815.590, 669350.670 70...",3761,771358


In [6]:
# stworzenie multipoligonów odpowiadającym zabudowom konkretnych numerów miejscowości
budynki_merged = budynki_grouped.groupby('SIMC_id')['geometry'].apply(lambda x: MultiPolygon(list(x)))
budynki_merged

SIMC_id
113419     MULTIPOLYGON (((576742.340 600926.110, 576753....
113431     MULTIPOLYGON (((579247.390 591836.840, 579249....
113448     MULTIPOLYGON (((576212.880 603469.120, 576212....
113454     MULTIPOLYGON (((578129.160 615174.410, 578134....
113460     MULTIPOLYGON (((572704.220 592411.230, 572710....
                                 ...                        
1063126    MULTIPOLYGON (((568038.790 682086.680, 568029....
1064491    MULTIPOLYGON (((558071.060 719577.550, 558078....
1064500    MULTIPOLYGON (((546468.940 721559.960, 546464....
1064516    MULTIPOLYGON (((661486.170 641861.350, 661487....
1067147    MULTIPOLYGON (((545328.540 719118.420, 545332....
Name: geometry, Length: 3763, dtype: geometry

In [7]:
budynki_merged_gdf = gpd.GeoDataFrame({'SIMC_id': budynki_merged.index, 'geometry': budynki_merged.values})
budynki_merged_gdf

,SIMC_id,geometry
0,113419,"MULTIPOLYGON (((576742.340 600926.110, 576753...."
1,113431,"MULTIPOLYGON (((579247.390 591836.840, 579249...."
2,113448,"MULTIPOLYGON (((576212.880 603469.120, 576212...."
3,113454,"MULTIPOLYGON (((578129.160 615174.410, 578134...."
4,113460,"MULTIPOLYGON (((572704.220 592411.230, 572710...."
...,...,...
3758,1063126,"MULTIPOLYGON (((568038.790 682086.680, 568029...."
3759,1064491,"MULTIPOLYGON (((558071.060 719577.550, 558078...."
3760,1064500,"MULTIPOLYGON (((546468.940 721559.960, 546464...."
3761,1064516,"MULTIPOLYGON (((661486.170 641861.350, 661487...."


In [8]:
# zapis do pliku takiej warstwy pomocniczej
# budynki_merged_gdf.to_file('28/warminskie_budynki_grouped.shp')

ekstraktowanie cech, korzystamy na razie z warstwy poligon a nie multipoligon

In [9]:
# liczba budynków
liczba_budynkow = budynki_grouped['SIMC_id'].value_counts().reset_index()
liczba_budynkow

,SIMC_id,count
0,845654,1600
1,125061,1578
2,486296,1243
3,758240,1224
4,472963,1137
...,...,...
3758,474761,1
3759,158385,1
3760,474040,1
3761,471350,1


In [10]:
# łączna powierzchnia zabudowy
powierzchnia_budynkow = budynki_grouped.groupby('SIMC_id')['area'].sum().reset_index()
powierzchnia_budynkow

,SIMC_id,area
0,113419,83569.42
1,113431,3383.35
2,113448,30639.25
3,113454,17698.85
4,113460,5193.56
...,...,...
3758,1063126,1601.74
3759,1064491,16319.27
3760,1064500,311.89
3761,1064516,430.60


In [11]:
# średnia liczba kondygnacji
srednia_liczba_kondygnacji = budynki_grouped.groupby('SIMC_id')['LKOND'].mean().reset_index()
srednia_liczba_kondygnacji

,SIMC_id,LKOND
0,113419,1.583784
1,113431,1.250000
2,113448,1.606987
3,113454,1.278351
4,113460,1.145833
...,...,...
3758,1063126,1.636364
3759,1064491,1.423077
3760,1064500,1.333333
3761,1064516,1.428571


In [12]:
# informacyjne wyświetlenie jakie są typy budynków
budynki_grouped['FUNOGBUD'].unique()

array(['budynkiMieszkalneJednorodzinne', 'budynekGospodarstwaRolnego',
       'budynkiOTrzechIWiecejMieszkaniach', 'budynkiGarazy',
       'budynkiSzkolIInstytucjiBadawczych', 'budynkiHandlowoUslugowe',
       'zbiornikSilosIBudynkiMagazynowe', 'budynkiPrzemyslowe',
       'budynkiKultuReligijnego',
       'budynkiSzpitaliIZakladowOpiekiMedycznej',
       'budynkiZbiorowegoZamieszkania', 'pozostaleBudynkiNiemieszkalne',
       'budynkiBiurowe', 'ogolnodostepneObiektyKulturalne',
       'budynkiMuzeowIBibliotek', 'budynkiHoteli',
       'budynkiLacznosciDworcowITerminali', 'budynkiKulturyFizycznej',
       'budynkiODwochMieszkaniach',
       'budynkiZakwaterowaniaTurystycznegoPozostale', 'budynekZabytkowy'],
      dtype=object)

cechy związane z licznością konkretnych typów budynków

In [13]:
rodzaje = budynki_grouped.groupby(['SIMC_id', 'FUNOGBUD']).size().reset_index(name='Liczba')
rodzaje

,SIMC_id,FUNOGBUD,Liczba
0,113419,budynekGospodarstwaRolnego,146
1,113419,budynkiBiurowe,3
2,113419,budynkiGarazy,1
3,113419,budynkiHandlowoUslugowe,2
4,113419,budynkiHoteli,1
...,...,...,...
17609,1064500,budynkiMieszkalneJednorodzinne,1
17610,1064516,budynekGospodarstwaRolnego,5
17611,1064516,budynkiMieszkalneJednorodzinne,2
17612,1067147,budynekGospodarstwaRolnego,1


In [14]:
# zliczenie ile jest jakiego rodzaju budynku dla poszczególnych numerów miejscowości
rodzaje_budynkow = rodzaje.pivot(index='SIMC_id', columns='FUNOGBUD', values='Liczba').reset_index()
rodzaje_budynkow = rodzaje_budynkow.fillna(0).astype('Int64')
rodzaje_budynkow

FUNOGBUD,SIMC_id,budynekGospodarstwaRolnego,budynekZabytkowy,budynkiBiurowe,budynkiGarazy,budynkiHandlowoUslugowe,budynkiHoteli,budynkiKultuReligijnego,budynkiKulturyFizycznej,budynkiLacznosciDworcowITerminali,...,budynkiODwochMieszkaniach,budynkiOTrzechIWiecejMieszkaniach,budynkiPrzemyslowe,budynkiSzkolIInstytucjiBadawczych,budynkiSzpitaliIZakladowOpiekiMedycznej,budynkiZakwaterowaniaTurystycznegoPozostale,budynkiZbiorowegoZamieszkania,ogolnodostepneObiektyKulturalne,pozostaleBudynkiNiemieszkalne,zbiornikSilosIBudynkiMagazynowe
0,113419,146,0,3,1,2,1,1,0,0,...,0,2,12,3,0,0,1,0,1,14
1,113431,7,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
2,113448,104,0,2,0,2,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0
3,113454,59,0,1,1,0,0,0,0,0,...,2,5,0,0,0,0,0,1,1,2
4,113460,30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3758,1063126,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3759,1064491,29,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,2,0
3760,1064500,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3761,1064516,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


ekstraktowanie cech związanych ze stopniem rozproszenia zabudowy

In [15]:
# zamiana warstwy budynków z poligonowej na punktową - z pomocą centroidu poligonu
centroidy_budynkow = gpd.GeoDataFrame(data=budynki_grouped['SIMC_id'], geometry=budynki_grouped.centroid, crs=budynki_grouped.crs)
centroidy_budynkow

,SIMC_id,geometry
0,470332,POINT (610888.185 711185.365)
135,470332,POINT (611831.296 711887.857)
195,470332,POINT (611188.875 710785.120)
505,470332,POINT (611900.072 712057.030)
601,470332,POINT (610829.915 711050.100)
...,...,...
480985,755490,POINT (679710.232 719738.188)
482496,755490,POINT (679723.814 719573.960)
483727,755490,POINT (679707.597 719767.593)
477442,771358,POINT (669342.878 706807.736)


In [16]:
# centroidy= centroidy_budynkow.groupby('SIMC_id')['geometry'].apply(lambda x: list(x))
# centroidy

In [17]:
# zamiana na multipoint
centroidy_merged = centroidy_budynkow.groupby('SIMC_id')['geometry'].apply(lambda x: MultiPoint(list(x))).reset_index()
centroidy_merged

,SIMC_id,geometry
0,113419,"MULTIPOINT (576747.864 600922.497, 576250.830 ..."
1,113431,"MULTIPOINT (579243.851 591835.299, 579323.945 ..."
2,113448,"MULTIPOINT (576205.658 603460.988, 576414.485 ..."
3,113454,"MULTIPOINT (578127.475 615167.425, 578472.345 ..."
4,113460,"MULTIPOINT (572707.255 592407.690, 572380.073 ..."
...,...,...
3758,1063126,"MULTIPOINT (568032.098 682087.088, 567972.906 ..."
3759,1064491,"MULTIPOINT (558076.015 719575.405, 558264.175 ..."
3760,1064500,"MULTIPOINT (546469.207 721565.172, 546309.944 ..."
3761,1064516,"MULTIPOINT (661485.062 641856.202, 661471.834 ..."


nna index - indeks analizy najbliższego sąsiada

In [30]:
# przygotowanie tabeli nna_indexes z danymi potrzebnymi do wyliczenia nna index dla każdego multipointa
nna_indexes = centroidy_merged.copy()
# area pobrana z granic miejscowosci - w m2
nna_indexes = nna_indexes.merge(granice_miejscowosci_z_adresacja[['SIMC_id', 'area']], how='left', on='SIMC_id')
nna_indexes.dropna(inplace=True)
nna_indexes.drop_duplicates(subset='SIMC_id', inplace=True)
nna_indexes

,SIMC_id,geometry,area
0,113419,"MULTIPOINT (576747.864 600922.497, 576250.830 ...",10911430.41
1,113431,"MULTIPOINT (579243.851 591835.299, 579323.945 ...",3102380.56
2,113448,"MULTIPOINT (576205.658 603460.988, 576414.485 ...",11637916.16
3,113454,"MULTIPOINT (578127.475 615167.425, 578472.345 ...",6158587.19
4,113460,"MULTIPOINT (572707.255 592407.690, 572380.073 ...",1709945.42
...,...,...,...
3767,1063126,"MULTIPOINT (568032.098 682087.088, 567972.906 ...",860289.77
3768,1064491,"MULTIPOINT (558076.015 719575.405, 558264.175 ...",205270.34
3769,1064500,"MULTIPOINT (546469.207 721565.172, 546309.944 ...",38286.56
3770,1064516,"MULTIPOINT (661485.062 641856.202, 661471.834 ...",5173336.90


In [31]:
# funkcja do wyznaczania macierzy odległości
def compute_distances_matrix(point_list):
    # print(point_list)
    # zamiana współrzędnych na postać, którą akceptuje numpy
    coordinates = [(point.x, point.y) for point in point_list]
    # macierz odległości
    return cdist(coordinates, coordinates)

# funkcja do obliczania nna_index
def compute_nna_index(multipoint, area):
    # zamiana multipoint na listę punktów
    point_list = list(multipoint.geoms)
    distances_matrix = compute_distances_matrix(point_list)
    # zmiana wartości na diagonali z 0 (odległość punktu od jego samego) na największą wartość z danej kolumny
    np.fill_diagonal(distances_matrix, np.max(distances_matrix, axis=0))
    # print(distances_matrix)
    # obliczenie sumy odległości do najbliższego sąsiada dla każdego punktu
    shortest_distance_sum = np.sum(np.min(distances_matrix, axis=1))
    # print(shortest_distance_sum)
    # liczba punktów poddawanych analizie
    point_numer = len(point_list)
    # print(point_numer)
    # wzór na składnik d0
    d0 = shortest_distance_sum/point_numer
    # print(d0)
    # wzór na składnik de
    de = 0.5/np.sqrt(point_numer/area)
    # print(de)
    # print(d0/de)
    # zwrócenie nna index
    return d0/de

# wskaznik = compute_nna_index(nna_indexes.loc[0]['geometry'], nna_indexes.loc[0]['area'])
# wskaznik
# obliczenie nna index dla każdego skupiska multipoint, które odpowiada rozmieszczeniu budynków
nna_indexes['nna_index'] = [compute_nna_index(multipoint, area) for multipoint, area in zip(nna_indexes['geometry'], nna_indexes['area'])]
nna_indexes

,SIMC_id,geometry,area,nna_index
0,113419,"MULTIPOINT (576747.864 600922.497, 576250.830 ...",10911430.41,0.349610
1,113431,"MULTIPOINT (579243.851 591835.299, 579323.945 ...",3102380.56,0.196046
2,113448,"MULTIPOINT (576205.658 603460.988, 576414.485 ...",11637916.16,0.212606
3,113454,"MULTIPOINT (578127.475 615167.425, 578472.345 ...",6158587.19,0.235016
4,113460,"MULTIPOINT (572707.255 592407.690, 572380.073 ...",1709945.42,0.336787
...,...,...,...,...
3767,1063126,"MULTIPOINT (568032.098 682087.088, 567972.906 ...",860289.77,0.200409
3768,1064491,"MULTIPOINT (558076.015 719575.405, 558264.175 ...",205270.34,0.917561
3769,1064500,"MULTIPOINT (546469.207 721565.172, 546309.944 ...",38286.56,1.179343
3770,1064516,"MULTIPOINT (661485.062 641856.202, 661471.834 ...",5173336.90,0.028865


In [32]:
# tabela nna_index
nna_indexes_df = nna_indexes[['SIMC_id', 'nna_index']]
nna_indexes_df

,SIMC_id,nna_index
0,113419,0.349610
1,113431,0.196046
2,113448,0.212606
3,113454,0.235016
4,113460,0.336787
...,...,...
3767,1063126,0.200409
3768,1064491,0.917561
3769,1064500,1.179343
3770,1064516,0.028865


ekstraktowanie cech średniej odległości i współczynnika zmienności odległości dla centroidów budynków

In [33]:
# obliczenie sredniej odleglosci miedzy punktami dla kazdego multipointa
def compute_mean_distance(multipoint):
    point_list = list(multipoint.geoms)
    distances_matrix = compute_distances_matrix(point_list)
    return np.mean(distances_matrix)

mean_distance = centroidy_merged.copy()
# test_srednia = compute_mean_distance(mean_distance.loc[23]['geometry'])
# test_srednia
mean_distance['srednia_odleglosci'] = [compute_mean_distance(multipoint) for multipoint in mean_distance['geometry']]
mean_distance

,SIMC_id,geometry,srednia_odleglosci
0,113419,"MULTIPOINT (576747.864 600922.497, 576250.830 ...",1066.709097
1,113431,"MULTIPOINT (579243.851 591835.299, 579323.945 ...",247.277883
2,113448,"MULTIPOINT (576205.658 603460.988, 576414.485 ...",497.217195
3,113454,"MULTIPOINT (578127.475 615167.425, 578472.345 ...",486.756717
4,113460,"MULTIPOINT (572707.255 592407.690, 572380.073 ...",441.816209
...,...,...,...
3758,1063126,"MULTIPOINT (568032.098 682087.088, 567972.906 ...",137.338223
3759,1064491,"MULTIPOINT (558076.015 719575.405, 558264.175 ...",193.946638
3760,1064500,"MULTIPOINT (546469.207 721565.172, 546309.944 ...",78.110162
3761,1064516,"MULTIPOINT (661485.062 641856.202, 661471.834 ...",51.428196


In [34]:
# obliczenie odchylenia standardowego odleglosci miedzy punktami dla kazdego multipointa
def compute_std(multipoint):
    point_list = list(multipoint.geoms)
    distances_matrix = compute_distances_matrix(point_list)
    return np.std(distances_matrix)

std_distance = mean_distance.copy()
# test_std = compute_std(std_distance.loc[23]['geometry'])
# test_std
std_distance['odchylenie_standardowe'] = [compute_std(multipoint) for multipoint in std_distance['geometry']]
std_distance

,SIMC_id,geometry,srednia_odleglosci,odchylenie_standardowe
0,113419,"MULTIPOINT (576747.864 600922.497, 576250.830 ...",1066.709097,907.926013
1,113431,"MULTIPOINT (579243.851 591835.299, 579323.945 ...",247.277883,170.333300
2,113448,"MULTIPOINT (576205.658 603460.988, 576414.485 ...",497.217195,401.666020
3,113454,"MULTIPOINT (578127.475 615167.425, 578472.345 ...",486.756717,374.794532
4,113460,"MULTIPOINT (572707.255 592407.690, 572380.073 ...",441.816209,284.333460
...,...,...,...,...
3758,1063126,"MULTIPOINT (568032.098 682087.088, 567972.906 ...",137.338223,94.242414
3759,1064491,"MULTIPOINT (558076.015 719575.405, 558264.175 ...",193.946638,128.815451
3760,1064500,"MULTIPOINT (546469.207 721565.172, 546309.944 ...",78.110162,77.477626
3761,1064516,"MULTIPOINT (661485.062 641856.202, 661471.834 ...",51.428196,40.195020


In [37]:
# obliczenie współczynnika zmienności średniej odległości dla kazdego multipointa
std_distance['wspolczynnik_zmiennosci'] = (std_distance['odchylenie_standardowe']/std_distance['srednia_odleglosci'])*100
std_distance

,SIMC_id,geometry,srednia_odleglosci,odchylenie_standardowe,wspolczynnik_zmiennosci
0,113419,"MULTIPOINT (576747.864 600922.497, 576250.830 ...",1066.709097,907.926013,85.114678
1,113431,"MULTIPOINT (579243.851 591835.299, 579323.945 ...",247.277883,170.333300,68.883354
2,113448,"MULTIPOINT (576205.658 603460.988, 576414.485 ...",497.217195,401.666020,80.782810
3,113454,"MULTIPOINT (578127.475 615167.425, 578472.345 ...",486.756717,374.794532,76.998328
4,113460,"MULTIPOINT (572707.255 592407.690, 572380.073 ...",441.816209,284.333460,64.355597
...,...,...,...,...,...
3758,1063126,"MULTIPOINT (568032.098 682087.088, 567972.906 ...",137.338223,94.242414,68.620673
3759,1064491,"MULTIPOINT (558076.015 719575.405, 558264.175 ...",193.946638,128.815451,66.417986
3760,1064500,"MULTIPOINT (546469.207 721565.172, 546309.944 ...",78.110162,77.477626,99.190200
3761,1064516,"MULTIPOINT (661485.062 641856.202, 661471.834 ...",51.428196,40.195020,78.157554


In [45]:
std_distance_df = std_distance.drop('geometry', axis=1)
std_distance_df

,SIMC_id,srednia_odleglosci,odchylenie_standardowe,wspolczynnik_zmiennosci
0,113419,1066.709097,907.926013,85.114678
1,113431,247.277883,170.333300,68.883354
2,113448,497.217195,401.666020,80.782810
3,113454,486.756717,374.794532,76.998328
4,113460,441.816209,284.333460,64.355597
...,...,...,...,...
3758,1063126,137.338223,94.242414,68.620673
3759,1064491,193.946638,128.815451,66.417986
3760,1064500,78.110162,77.477626,99.190200
3761,1064516,51.428196,40.195020,78.157554


ekstraktowanie cechy minimalnej i maksymalnej odległości między centroidami budynków dla każdej miejscowości oraz ich rozstępu

In [38]:
min_max_distance = centroidy_merged.copy()
min_max_distance

,SIMC_id,geometry
0,113419,"MULTIPOINT (576747.864 600922.497, 576250.830 ..."
1,113431,"MULTIPOINT (579243.851 591835.299, 579323.945 ..."
2,113448,"MULTIPOINT (576205.658 603460.988, 576414.485 ..."
3,113454,"MULTIPOINT (578127.475 615167.425, 578472.345 ..."
4,113460,"MULTIPOINT (572707.255 592407.690, 572380.073 ..."
...,...,...
3758,1063126,"MULTIPOINT (568032.098 682087.088, 567972.906 ..."
3759,1064491,"MULTIPOINT (558076.015 719575.405, 558264.175 ..."
3760,1064500,"MULTIPOINT (546469.207 721565.172, 546309.944 ..."
3761,1064516,"MULTIPOINT (661485.062 641856.202, 661471.834 ..."


In [40]:
# obliczenie min odleglosci miedzy punktami dla kazdego multipointa
def compute_min(multipoint):
    point_list = list(multipoint.geoms)
    distances_matrix = compute_distances_matrix(point_list)
    np.fill_diagonal(distances_matrix, np.max(distances_matrix, axis=0))
    return np.min(distances_matrix)

# obliczenie max odleglosci miedzy punktami dla kazdego multipointa
def compute_max(multipoint):
    point_list = list(multipoint.geoms)
    distances_matrix = compute_distances_matrix(point_list)
    return np.max(distances_matrix)

# obliczenie ptp odleglosci miedzy punktami dla kazdego multipointa
def compute_ptp(multipoint):
    point_list = list(multipoint.geoms)
    distances_matrix = compute_distances_matrix(point_list)
    return np.ptp(distances_matrix)

min_max_distance['min_odleglosc'] = [compute_min(multipoint) for multipoint in min_max_distance['geometry']]
min_max_distance['max_odleglosc'] = [compute_max(multipoint) for multipoint in min_max_distance['geometry']]
min_max_distance['min_max_rozstep'] = [compute_ptp(multipoint) for multipoint in min_max_distance['geometry']]
min_max_distance

,SIMC_id,geometry,min_odleglosc,max_odleglosc,min_max_rozstep
0,113419,"MULTIPOINT (576747.864 600922.497, 576250.830 ...",5.595754,4490.197390,4490.197390
1,113431,"MULTIPOINT (579243.851 591835.299, 579323.945 ...",10.068545,520.223242,520.223242
2,113448,"MULTIPOINT (576205.658 603460.988, 576414.485 ...",6.621034,2849.556375,2849.556375
3,113454,"MULTIPOINT (578127.475 615167.425, 578472.345 ...",5.007587,2204.703636,2204.703636
4,113460,"MULTIPOINT (572707.255 592407.690, 572380.073 ...",5.033175,1253.942402,1253.942402
...,...,...,...,...,...
3758,1063126,"MULTIPOINT (568032.098 682087.088, 567972.906 ...",12.877561,304.774513,304.774513
3759,1064491,"MULTIPOINT (558076.015 719575.405, 558264.175 ...",5.625767,614.792574,614.792574
3760,1064500,"MULTIPOINT (546469.207 721565.172, 546309.944 ...",24.087480,175.737876,175.737876
3761,1064516,"MULTIPOINT (661485.062 641856.202, 661471.834 ...",11.037517,111.790464,111.790464


In [46]:
min_max_distance_df = min_max_distance.drop('geometry', axis=1)
min_max_distance_df

,SIMC_id,min_odleglosc,max_odleglosc,min_max_rozstep
0,113419,5.595754,4490.197390,4490.197390
1,113431,10.068545,520.223242,520.223242
2,113448,6.621034,2849.556375,2849.556375
3,113454,5.007587,2204.703636,2204.703636
4,113460,5.033175,1253.942402,1253.942402
...,...,...,...,...
3758,1063126,12.877561,304.774513,304.774513
3759,1064491,5.625767,614.792574,614.792574
3760,1064500,24.087480,175.737876,175.737876
3761,1064516,11.037517,111.790464,111.790464


połączenie wszystkich wyekstraktowanych cech zabudowy

In [43]:
zabudowa_merged = centroidy_merged.copy()
zabudowa_merged

,SIMC_id,geometry
0,113419,"MULTIPOINT (576747.864 600922.497, 576250.830 ..."
1,113431,"MULTIPOINT (579243.851 591835.299, 579323.945 ..."
2,113448,"MULTIPOINT (576205.658 603460.988, 576414.485 ..."
3,113454,"MULTIPOINT (578127.475 615167.425, 578472.345 ..."
4,113460,"MULTIPOINT (572707.255 592407.690, 572380.073 ..."
...,...,...
3758,1063126,"MULTIPOINT (568032.098 682087.088, 567972.906 ..."
3759,1064491,"MULTIPOINT (558076.015 719575.405, 558264.175 ..."
3760,1064500,"MULTIPOINT (546469.207 721565.172, 546309.944 ..."
3761,1064516,"MULTIPOINT (661485.062 641856.202, 661471.834 ..."


In [47]:
# merge wszystkich tabel po kolei po simc_id
tables_to_merge = [liczba_budynkow, powierzchnia_budynkow, srednia_liczba_kondygnacji, rodzaje_budynkow, nna_indexes_df, std_distance_df, min_max_distance_df]
for table in tables_to_merge:
    zabudowa_merged = zabudowa_merged.merge(table, how='left', on='SIMC_id')
    zabudowa_merged.drop_duplicates(inplace=True)
zabudowa_merged

,SIMC_id,geometry,count,area,LKOND,budynekGospodarstwaRolnego,budynekZabytkowy,budynkiBiurowe,budynkiGarazy,budynkiHandlowoUslugowe,...,ogolnodostepneObiektyKulturalne,pozostaleBudynkiNiemieszkalne,zbiornikSilosIBudynkiMagazynowe,nna_index,srednia_odleglosci,odchylenie_standardowe,wspolczynnik_zmiennosci,min_odleglosc,max_odleglosc,min_max_rozstep
0,113419,"MULTIPOINT (576747.864 600922.497, 576250.830 ...",371,83569.42,1.583784,146,0,3,1,2,...,0,1,14,0.349610,1066.709097,907.926013,85.114678,5.595754,4490.197390,4490.197390
1,113431,"MULTIPOINT (579243.851 591835.299, 579323.945 ...",12,3383.35,1.250000,7,0,0,0,0,...,0,0,0,0.196046,247.277883,170.333300,68.883354,10.068545,520.223242,520.223242
2,113448,"MULTIPOINT (576205.658 603460.988, 576414.485 ...",229,30639.25,1.606987,104,0,2,0,2,...,0,0,0,0.212606,497.217195,401.666020,80.782810,6.621034,2849.556375,2849.556375
3,113454,"MULTIPOINT (578127.475 615167.425, 578472.345 ...",97,17698.85,1.278351,59,0,1,1,0,...,1,1,2,0.235016,486.756717,374.794532,76.998328,5.007587,2204.703636,2204.703636
4,113460,"MULTIPOINT (572707.255 592407.690, 572380.073 ...",48,5193.56,1.145833,30,0,0,0,0,...,0,0,0,0.336787,441.816209,284.333460,64.355597,5.033175,1253.942402,1253.942402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3758,1063126,"MULTIPOINT (568032.098 682087.088, 567972.906 ...",22,1601.74,1.636364,7,0,0,0,0,...,0,0,0,0.200409,137.338223,94.242414,68.620673,12.877561,304.774513,304.774513
3759,1064491,"MULTIPOINT (558076.015 719575.405, 558264.175 ...",52,16319.27,1.423077,29,0,0,0,0,...,0,2,0,0.917561,193.946638,128.815451,66.417986,5.625767,614.792574,614.792574
3760,1064500,"MULTIPOINT (546469.207 721565.172, 546309.944 ...",3,311.89,1.333333,2,0,0,0,0,...,0,0,0,1.179343,78.110162,77.477626,99.190200,24.087480,175.737876,175.737876
3761,1064516,"MULTIPOINT (661485.062 641856.202, 661471.834 ...",7,430.60,1.428571,5,0,0,0,0,...,0,0,0,0.028865,51.428196,40.195020,78.157554,11.037517,111.790464,111.790464


In [49]:
# zmiana nazw niektórych kolumn na bardziej czytelne i wyrzucenie geometrii centroidów
zabudowa_merged.rename(columns={'count': 'liczba_budynkow', 'area': 'powierzchnia_budynkow', 'LKOND': 'srednia_liczba_kondygnacji'}, inplace=True)
zabudowa_merged_df = zabudowa_merged.drop('geometry', axis=1)
zabudowa_merged_df

,SIMC_id,liczba_budynkow,powierzchnia_budynkow,srednia_liczba_kondygnacji,budynekGospodarstwaRolnego,budynekZabytkowy,budynkiBiurowe,budynkiGarazy,budynkiHandlowoUslugowe,budynkiHoteli,...,ogolnodostepneObiektyKulturalne,pozostaleBudynkiNiemieszkalne,zbiornikSilosIBudynkiMagazynowe,nna_index,srednia_odleglosci,odchylenie_standardowe,wspolczynnik_zmiennosci,min_odleglosc,max_odleglosc,min_max_rozstep
0,113419,371,83569.42,1.583784,146,0,3,1,2,1,...,0,1,14,0.349610,1066.709097,907.926013,85.114678,5.595754,4490.197390,4490.197390
1,113431,12,3383.35,1.250000,7,0,0,0,0,0,...,0,0,0,0.196046,247.277883,170.333300,68.883354,10.068545,520.223242,520.223242
2,113448,229,30639.25,1.606987,104,0,2,0,2,0,...,0,0,0,0.212606,497.217195,401.666020,80.782810,6.621034,2849.556375,2849.556375
3,113454,97,17698.85,1.278351,59,0,1,1,0,0,...,1,1,2,0.235016,486.756717,374.794532,76.998328,5.007587,2204.703636,2204.703636
4,113460,48,5193.56,1.145833,30,0,0,0,0,0,...,0,0,0,0.336787,441.816209,284.333460,64.355597,5.033175,1253.942402,1253.942402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3758,1063126,22,1601.74,1.636364,7,0,0,0,0,0,...,0,0,0,0.200409,137.338223,94.242414,68.620673,12.877561,304.774513,304.774513
3759,1064491,52,16319.27,1.423077,29,0,0,0,0,0,...,0,2,0,0.917561,193.946638,128.815451,66.417986,5.625767,614.792574,614.792574
3760,1064500,3,311.89,1.333333,2,0,0,0,0,0,...,0,0,0,1.179343,78.110162,77.477626,99.190200,24.087480,175.737876,175.737876
3761,1064516,7,430.60,1.428571,5,0,0,0,0,0,...,0,0,0,0.028865,51.428196,40.195020,78.157554,11.037517,111.790464,111.790464


In [52]:
# zbadanie, czy występują wartości null albo nan
na = zabudowa_merged_df[zabudowa_merged_df.isnull().any(axis=1)]
na
# Wartości nan występują dla 13 miejscowości dla współczynnika zmienności. Są to miejscowości z jednym budynkiem, więc współczynnik o wartości nan wyniknął z podzielenia przez 0 (odchylenie/średnią odległości, co daje 0/0). Zostanie on zastąpiony wartością 0, gdyż interpretacja tego współczynnika to "im wyższy współczynnik zmienności, tym niższa regularność". Wartość 0 to najmniejsza możliwa, więc oznaczająca wysoki poziom regularności - jeden budynek to zdecydowana regularność

,SIMC_id,liczba_budynkow,powierzchnia_budynkow,srednia_liczba_kondygnacji,budynekGospodarstwaRolnego,budynekZabytkowy,budynkiBiurowe,budynkiGarazy,budynkiHandlowoUslugowe,budynkiHoteli,...,ogolnodostepneObiektyKulturalne,pozostaleBudynkiNiemieszkalne,zbiornikSilosIBudynkiMagazynowe,nna_index,srednia_odleglosci,odchylenie_standardowe,wspolczynnik_zmiennosci,min_odleglosc,max_odleglosc,min_max_rozstep
455,154625,1,273.40,2.0,0,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
600,158385,1,173.67,2.0,0,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
648,468973,1,120.19,1.0,0,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
869,471350,1,132.47,2.0,0,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
898,471691,1,113.44,1.0,0,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
1030,473170,1,252.50,2.0,0,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
1114,474040,1,255.45,1.0,0,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
1175,474761,1,100.15,2.0,0,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
1460,477802,1,131.82,2.0,0,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
1485,478055,1,122.38,2.0,0,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,NaN,0.0,0.0,0.0


In [53]:
zabudowa_merged_df.fillna(0, inplace=True)
zabudowa_merged_df

,SIMC_id,liczba_budynkow,powierzchnia_budynkow,srednia_liczba_kondygnacji,budynekGospodarstwaRolnego,budynekZabytkowy,budynkiBiurowe,budynkiGarazy,budynkiHandlowoUslugowe,budynkiHoteli,...,ogolnodostepneObiektyKulturalne,pozostaleBudynkiNiemieszkalne,zbiornikSilosIBudynkiMagazynowe,nna_index,srednia_odleglosci,odchylenie_standardowe,wspolczynnik_zmiennosci,min_odleglosc,max_odleglosc,min_max_rozstep
0,113419,371,83569.42,1.583784,146,0,3,1,2,1,...,0,1,14,0.349610,1066.709097,907.926013,85.114678,5.595754,4490.197390,4490.197390
1,113431,12,3383.35,1.250000,7,0,0,0,0,0,...,0,0,0,0.196046,247.277883,170.333300,68.883354,10.068545,520.223242,520.223242
2,113448,229,30639.25,1.606987,104,0,2,0,2,0,...,0,0,0,0.212606,497.217195,401.666020,80.782810,6.621034,2849.556375,2849.556375
3,113454,97,17698.85,1.278351,59,0,1,1,0,0,...,1,1,2,0.235016,486.756717,374.794532,76.998328,5.007587,2204.703636,2204.703636
4,113460,48,5193.56,1.145833,30,0,0,0,0,0,...,0,0,0,0.336787,441.816209,284.333460,64.355597,5.033175,1253.942402,1253.942402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3758,1063126,22,1601.74,1.636364,7,0,0,0,0,0,...,0,0,0,0.200409,137.338223,94.242414,68.620673,12.877561,304.774513,304.774513
3759,1064491,52,16319.27,1.423077,29,0,0,0,0,0,...,0,2,0,0.917561,193.946638,128.815451,66.417986,5.625767,614.792574,614.792574
3760,1064500,3,311.89,1.333333,2,0,0,0,0,0,...,0,0,0,1.179343,78.110162,77.477626,99.190200,24.087480,175.737876,175.737876
3761,1064516,7,430.60,1.428571,5,0,0,0,0,0,...,0,0,0,0.028865,51.428196,40.195020,78.157554,11.037517,111.790464,111.790464


In [55]:
# zapis do csv
# zabudowa_merged_df.to_csv("28/analiza_zabudowy_warminskie.csv", sep=';', header=True, index=False)